# SINTA Web Scraping - ETL dengan Hadoop

Notebook ini akan:
1. Clone repository dari GitHub
2. Install dependencies
3. Install dan setup Hadoop
4. Menjalankan proses ETL

---


## Step 1: Clone Repository


In [ ]:
# Clone repository
!git clone https://github.com/Herutriana44/sinta_web_scraping.git

# Pindah ke direktori project
import os
os.chdir('sinta_web_scraping')

print("✅ Repository berhasil di-clone")
print(f"📁 Current directory: {os.getcwd()}")


## Step 2: Install Dependencies Python


In [ ]:
# Install build dependencies terlebih dahulu (untuk hdfs3)
print("📦 Menginstall build dependencies...")
!apt-get update -qq
!apt-get install -y -qq build-essential libssl-dev libffi-dev python3-dev > /dev/null 2>&1

# Install dependencies Python dasar
print("📦 Menginstall dependencies Python dasar...")
!pip install -q selenium webdriver-manager bs4 beautifulsoup4

# Coba install hdfs3 dengan build dependencies
print("\n📦 Mencoba install hdfs3...")
import subprocess
result = subprocess.run(
    ['pip', 'install', '-q', 'hdfs3'],
    capture_output=True,
    text=True
)

# Cek apakah berhasil
try:
    import hdfs3
    from hdfs3 import HDFileSystem
    print("✅ hdfs3 berhasil diinstall dan dapat diimport")
    HDFS3_AVAILABLE = True
except ImportError:
    print("⚠️ hdfs3 gagal diinstall atau tidak dapat diimport")
    if result.stderr:
        print(f"   Error: {result.stderr[:200]}...")  # Tampilkan sebagian error
    print("📝 Notebook akan menggunakan alternatif: subprocess wrapper untuk HDFS")
    HDFS3_AVAILABLE = False

print("\n✅ Dependencies Python berhasil diinstall")


## Step 2.1: Setup Alternatif HDFS (Jika hdfs3 Gagal)


In [ ]:
# Jika hdfs3 tidak tersedia, kita akan menggunakan subprocess untuk HDFS commands
# Modifikasi sementara file ETL untuk mendukung alternatif

import os
import subprocess
from pathlib import Path

# Cek apakah hdfs3 tersedia
try:
    from hdfs3 import HDFileSystem
    HDFS3_AVAILABLE = True
    print("✅ hdfs3 tersedia, akan menggunakan library hdfs3")
except ImportError:
    HDFS3_AVAILABLE = False
    print("⚠️ hdfs3 tidak tersedia, akan menggunakan subprocess untuk HDFS")
    
    # Buat wrapper untuk HDFS menggunakan subprocess
    # Ini akan digunakan jika hdfs3 tidak tersedia
    print("📝 Membuat wrapper HDFS menggunakan subprocess...")
    
    # File wrapper akan dibuat setelah Hadoop diinstall
    print("✅ Wrapper akan dibuat setelah Hadoop diinstall")

print(f"\n📊 Status: HDFS3_AVAILABLE = {HDFS3_AVAILABLE}")


## Step 3: Install Java (Required untuk Hadoop)


In [ ]:
# Install Java JDK 8 (required untuk Hadoop)
!apt-get update -qq
!apt-get install -y -qq openjdk-8-jdk > /dev/null 2>&1

# Set JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

# Verify Java installation
!java -version

print("\n✅ Java berhasil diinstall")


## Step 4: Install dan Setup Hadoop


In [ ]:
# Download Hadoop 3.3.6
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

# Extract Hadoop
!tar -xzf hadoop-3.3.6.tar.gz

# Set environment variables
import os
os.environ['HADOOP_HOME'] = os.path.join(os.getcwd(), 'hadoop-3.3.6')
os.environ['HADOOP_CONF_DIR'] = os.path.join(os.environ['HADOOP_HOME'], 'etc/hadoop')
os.environ['PATH'] = os.path.join(os.environ['HADOOP_HOME'], 'bin') + ':' + \
                     os.path.join(os.environ['HADOOP_HOME'], 'sbin') + ':' + os.environ['PATH']

print("✅ Hadoop berhasil di-download dan di-extract")
print(f"📁 HADOOP_HOME: {os.environ['HADOOP_HOME']}")


## Step 5: Konfigurasi Hadoop (Pseudo-Distributed Mode)


In [ ]:
import os

# Path ke direktori konfigurasi Hadoop
hadoop_conf_dir = os.path.join(os.environ['HADOOP_HOME'], 'etc/hadoop')

# 1. Konfigurasi core-site.xml
core_site_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>'''

with open(os.path.join(hadoop_conf_dir, 'core-site.xml'), 'w') as f:
    f.write(core_site_xml)

# 2. Konfigurasi hdfs-site.xml
hdfs_site_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
    <property>
        <name>dfs.namenode.name.dir</name>
        <value>file:///tmp/hadoop-${user.name}/dfs/name</value>
    </property>
    <property>
        <name>dfs.datanode.data.dir</name>
        <value>file:///tmp/hadoop-${user.name}/dfs/data</value>
    </property>
</configuration>'''

with open(os.path.join(hadoop_conf_dir, 'hdfs-site.xml'), 'w') as f:
    f.write(hdfs_site_xml)

# 3. Konfigurasi mapred-site.xml
mapred_site_xml = '''<?xml version="1.0"?>
<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
</configuration>'''

with open(os.path.join(hadoop_conf_dir, 'mapred-site.xml'), 'w') as f:
    f.write(mapred_site_xml)

# 4. Konfigurasi yarn-site.xml
yarn_site_xml = '''<?xml version="1.0"?>
<configuration>
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
</configuration>'''

with open(os.path.join(hadoop_conf_dir, 'yarn-site.xml'), 'w') as f:
    f.write(yarn_site_xml)

# 5. Set JAVA_HOME di hadoop-env.sh
hadoop_env_sh = f'''export JAVA_HOME={os.environ['JAVA_HOME']}
export HADOOP_HOME={os.environ['HADOOP_HOME']}
export HADOOP_CONF_DIR={os.environ['HADOOP_CONF_DIR']}
'''

with open(os.path.join(hadoop_conf_dir, 'hadoop-env.sh'), 'a') as f:
    f.write(hadoop_env_sh)

print("✅ Konfigurasi Hadoop selesai")


## Step 6: Setup SSH (Required untuk Hadoop)


In [ ]:
# Install SSH server
!apt-get install -y -qq openssh-server > /dev/null 2>&1

# Setup SSH untuk Hadoop
!ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa > /dev/null 2>&1
!cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
!chmod 0600 ~/.ssh/authorized_keys

# Start SSH service
!service ssh start > /dev/null 2>&1

# Test SSH connection
!ssh-keyscan -H localhost >> ~/.ssh/known_hosts 2>/dev/null

print("✅ SSH berhasil dikonfigurasi")


## Step 7: Format HDFS dan Start Hadoop Services


In [ ]:
import os
import subprocess
import time

# Format HDFS (hanya perlu dilakukan sekali, tapi aman untuk diulang)
print("📝 Formatting HDFS...")
format_result = subprocess.run(
    ['hdfs', 'namenode', '-format', '-force', '-nonInteractive'],
    env=os.environ,
    capture_output=True,
    text=True
)

if format_result.returncode == 0:
    print("✅ HDFS berhasil di-format")
else:
    print(f"⚠️ Format HDFS: {format_result.stderr}")

# Start NameNode
print("\n🚀 Starting NameNode...")
namenode_process = subprocess.Popen(
    ['hdfs', '--daemon', 'start', 'namenode'],
    env=os.environ
)
time.sleep(5)

# Start DataNode
print("🚀 Starting DataNode...")
datanode_process = subprocess.Popen(
    ['hdfs', '--daemon', 'start', 'datanode'],
    env=os.environ
)
time.sleep(5)

# Check status
print("\n📊 Checking Hadoop services status...")
!jps

print("\n✅ Hadoop services sudah berjalan")
print("🌐 NameNode Web UI: http://localhost:9870")
print("📁 HDFS Path: hdfs://localhost:9000")


## Step 8: Verifikasi HDFS dan Buat Direktori


In [ ]:
import subprocess
import os

# Tunggu beberapa detik untuk memastikan HDFS siap
import time
time.sleep(3)

# Test HDFS dengan membuat direktori
print("📁 Membuat direktori di HDFS...")

# Buat direktori untuk data SINTA
result = subprocess.run(
    ['hdfs', 'dfs', '-mkdir', '-p', '/user/sinta/journals'],
    env=os.environ,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Direktori HDFS berhasil dibuat: /user/sinta/journals")
else:
    print(f"⚠️ {result.stderr}")

# List direktori HDFS
print("\n📂 Isi direktori HDFS:")
!hdfs dfs -ls -R /user/sinta/

print("\n✅ HDFS siap digunakan!")


## Step 9: Jalankan ETL Process


## Step 8.1: Buat HDFS Wrapper (Jika hdfs3 Tidak Tersedia)


In [ ]:
# Buat wrapper HDFS menggunakan subprocess jika hdfs3 tidak tersedia
import os
import subprocess
from pathlib import Path

# Cek apakah hdfs3 tersedia
try:
    from hdfs3 import HDFileSystem
    HDFS3_AVAILABLE = True
    print("✅ hdfs3 tersedia, akan menggunakan library hdfs3")
except ImportError:
    HDFS3_AVAILABLE = False
    print("⚠️ hdfs3 tidak tersedia, akan membuat wrapper menggunakan subprocess...")
    
if not HDFS3_AVAILABLE:
    # Buat file helper untuk HDFS operations
    hdfs_helper_code = '''"""
HDFS Helper menggunakan subprocess (alternatif jika hdfs3 tidak tersedia)
"""
import os
import subprocess
import tempfile
from pathlib import Path

class HDFileSystemSubprocess:
    """Wrapper untuk HDFS menggunakan subprocess"""
    
    def __init__(self, host='localhost', port=9000, user=None):
        self.host = host
        self.port = port
        self.user = user
        self.env = os.environ.copy()
    
    def exists(self, path):
        """Cek apakah path ada di HDFS"""
        try:
            result = subprocess.run(
                ['hdfs', 'dfs', '-test', '-e', path],
                env=self.env,
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except:
            return False
    
    def makedirs(self, path):
        """Buat direktori di HDFS"""
        try:
            result = subprocess.run(
                ['hdfs', 'dfs', '-mkdir', '-p', path],
                env=self.env,
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except:
            return False
    
    def open(self, path, mode='rb'):
        """Open file di HDFS (untuk kompatibilitas dengan hdfs3)"""
        if 'w' in mode or 'a' in mode:
            return HDFSFileWriter(path, self.env)
        else:
            return HDFSFileReader(path, self.env)

class HDFSFileWriter:
    """File writer untuk HDFS menggunakan subprocess"""
    def __init__(self, hdfs_path, env):
        self.hdfs_path = hdfs_path
        self.env = env
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)
        self.temp_path = self.temp_file.name
        self.closed = False
    
    def write(self, data):
        if not self.closed:
            self.temp_file.write(data)
    
    def close(self):
        if not self.closed:
            self.temp_file.close()
            # Upload ke HDFS
            subprocess.run(
                ['hdfs', 'dfs', '-put', '-f', self.temp_path, self.hdfs_path],
                env=self.env,
                capture_output=True
            )
            # Hapus temp file
            try:
                os.unlink(self.temp_path)
            except:
                pass
            self.closed = True
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        self.close()

class HDFSFileReader:
    """File reader untuk HDFS menggunakan subprocess"""
    def __init__(self, hdfs_path, env):
        self.hdfs_path = hdfs_path
        self.env = env
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)
        self.temp_path = self.temp_file.name
        # Download dari HDFS
        subprocess.run(
            ['hdfs', 'dfs', '-get', hdfs_path, self.temp_path],
            env=self.env,
            capture_output=True
        )
        self.file = open(self.temp_path, 'rb')
        self.closed = False
    
    def read(self, size=-1):
        return self.file.read(size)
    
    def close(self):
        if not self.closed:
            self.file.close()
            try:
                os.unlink(self.temp_path)
            except:
                pass
            self.closed = True
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        self.close()
'''
    
    # Simpan helper code
    with open('hdfs_helper.py', 'w') as f:
        f.write(hdfs_helper_code)
    
    # Modifikasi sinta_journals_etl.py untuk menggunakan helper jika hdfs3 tidak tersedia
    print("📝 Memodifikasi sinta_journals_etl.py untuk menggunakan subprocess wrapper...")
    
    # Baca file ETL
    with open('sinta_journals_etl.py', 'r', encoding='utf-8') as f:
        etl_code = f.read()
    
    # Ganti import hdfs3 dengan fallback ke helper
    modified_import = '''# HDFS support (optional)
try:
    from hdfs3 import HDFileSystem
    HDFS_AVAILABLE = True
except ImportError:
    try:
        # Coba gunakan helper subprocess
        from hdfs_helper import HDFileSystemSubprocess as HDFileSystem
        HDFS_AVAILABLE = True
    except ImportError:
        HDFS_AVAILABLE = False
        logger.warning("⚠️ Library hdfs3 tidak tersedia. Fitur HDFS akan dinonaktifkan.")
        logger.warning("⚠️ Install dengan: pip install hdfs3")
'''
    
    # Ganti bagian import
    old_import = '''# HDFS support (optional)
try:
    from hdfs3 import HDFileSystem
    HDFS_AVAILABLE = True
except ImportError:
    HDFS_AVAILABLE = False
    logger.warning("⚠️ Library hdfs3 tidak tersedia. Fitur HDFS akan dinonaktifkan.")
    logger.warning("⚠️ Install dengan: pip install hdfs3")'''
    
    if old_import in etl_code:
        etl_code = etl_code.replace(old_import, modified_import)
        
        # Simpan file yang dimodifikasi
        with open('sinta_journals_etl.py', 'w', encoding='utf-8') as f:
            f.write(etl_code)
        
        print("✅ File ETL dimodifikasi untuk menggunakan subprocess wrapper")
        print("✅ HDFS wrapper menggunakan subprocess siap digunakan")
    else:
        print("⚠️ Tidak dapat menemukan bagian import hdfs3 untuk dimodifikasi")
        print("⚠️ Pastikan file sinta_journals_etl.py ada di direktori")


In [ ]:
# Import ETL module
from sinta_journals_etl import SINTAJournalsETL

# Inisialisasi ETL dengan HDFS
print("🔧 Menginisialisasi ETL dengan HDFS...")
etl = SINTAJournalsETL(
    input_folder="output_journals",
    output_folder="output_data",
    hdfs_enabled=True,
    hdfs_url="http://localhost:9870",
    hdfs_path="/user/sinta/journals",
    hdfs_user=None  # Gunakan user default
)

# Jalankan ETL process
print("\n🚀 Menjalankan proses ETL...")
etl.run(output_format='both', save_to_hdfs=True)

print("\n✅ ETL Process selesai!")


## Step 10: Verifikasi Hasil ETL


## Step 9.1: Cek Lokasi File di HDFS


In [ ]:
# Cek lokasi file di HDFS setelah ETL selesai
import subprocess
import os
from datetime import datetime

print("📁 Struktur Direktori HDFS:")
print("=" * 60)
print("Base Path: /user/sinta/journals")
print("Struktur: /user/sinta/journals/{YYYY}/{MM}/{DD}/{filename}")
print("=" * 60)

# Dapatkan tanggal hari ini untuk melihat folder
today = datetime.now().strftime("%Y/%m/%d")
hdfs_date_path = f"/user/sinta/journals/{today}"

print(f"\n📂 Path lengkap untuk hari ini: {hdfs_date_path}")
print("\n🔍 Listing semua file di HDFS:")

# List semua file di direktori base
result = subprocess.run(
    ['hdfs', 'dfs', '-ls', '-R', '/user/sinta/journals/'],
    env=os.environ,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(result.stdout)
else:
    print(f"⚠️ Error: {result.stderr}")
    print("\n💡 Coba list secara manual:")
    print(f"   !hdfs dfs -ls -R /user/sinta/journals/")

print("\n📝 Cara mengakses file di HDFS:")
print("=" * 60)
print("1. Melalui command line:")
print("   !hdfs dfs -cat /user/sinta/journals/YYYY/MM/DD/journals_data_*.csv")
print("\n2. Download dari HDFS ke lokal:")
print("   !hdfs dfs -get /user/sinta/journals/YYYY/MM/DD/journals_data_*.csv .")
print("\n3. Copy dari HDFS ke lokal:")
print("   !hdfs dfs -copyToLocal /user/sinta/journals/YYYY/MM/DD/journals_data_*.csv .")
print("=" * 60)


In [ ]:
import os
import json
from pathlib import Path

# Cek file output lokal
output_dir = Path('output_data')
if output_dir.exists():
    print("📁 File output lokal:")
    for file in sorted(output_dir.glob('*')):
        size = file.stat().st_size / 1024  # KB
        print(f"  - {file.name} ({size:.2f} KB)")
    
    # Tampilkan statistik jika ada
    stats_files = list(output_dir.glob('extraction_stats_*.json'))
    if stats_files:
        latest_stats = sorted(stats_files)[-1]
        with open(latest_stats, 'r') as f:
            stats = json.load(f)
        print("\n📊 Statistik Ekstraksi:")
        print(json.dumps(stats['statistics'], indent=2))

# Cek file di HDFS
print("\n📁 File di HDFS:")
!hdfs dfs -ls -R /user/sinta/journals/

print("\n✅ Verifikasi selesai!")


## Optional: Stop Hadoop Services


In [ ]:
# Stop Hadoop services jika diperlukan
import subprocess
import os

print("🛑 Menghentikan Hadoop services...")

subprocess.run(['hdfs', '--daemon', 'stop', 'datanode'], env=os.environ)
subprocess.run(['hdfs', '--daemon', 'stop', 'namenode'], env=os.environ)

print("✅ Hadoop services dihentikan")


## Alternatif: ETL Tanpa HDFS (Jika Hadoop Bermasalah)

Jika mengalami masalah dengan Hadoop di Colab, jalankan ETL tanpa HDFS:


In [ ]:
# Alternatif: ETL tanpa HDFS jika Hadoop bermasalah
from sinta_journals_etl import SINTAJournalsETL

print("🔧 Menginisialisasi ETL tanpa HDFS...")
etl_no_hdfs = SINTAJournalsETL(
    input_folder="output_journals",
    output_folder="output_data",
    hdfs_enabled=False  # Nonaktifkan HDFS
)

# Jalankan ETL process
print("\n🚀 Menjalankan proses ETL...")
etl_no_hdfs.run(output_format='both', save_to_hdfs=False)

print("\n✅ ETL Process selesai (tanpa HDFS)!")
